# Imports

In [1]:
import pandas as pd
import math

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# for file-management
import pickle as pkl
from pathlib import Path
import os.path
import sys

# Participant Infos

In [29]:
participants = list(range(1, 21))
smartphones = ["S3Mini", "S4", "N5X", "N6"]
# TODO remove
participants = list(range(1, 2))
smartphones = ["S3Mini"]
extension = 0
event_time_diff = 3

sensors = [
    "acc",
    "gyro",
    "ori",
    "grav",
    "mag",
    "rot"
]

tasks = [
    "points",
    "fitts"
]

file_names = sensors + tasks

In [44]:
%%time
for PID in participants:
    for smartphone in smartphones:
        print("processing - participant:", str(PID), "smartphone:", smartphone)

        # set screen resolution
        if smartphone == "N5X":
            pixels = {"width": 1080, "height": 1920}
        elif smartphone == "S3Mini":
            pixels = {"width": 480, "height": 800}
        elif smartphone == "S4":
            pixels = {"width": 1080, "height": 1920}
        elif smartphone == "N6":
            pixels = {"width": 1440, "height": 2560}

        # Read Files
        raw_data = dict()

        for file in file_names:
            file_path = str(Path.home()) + "/data/raw/fapra_imu-" +  str(PID) + "-" + file + "-" + smartphone + "-" + str(extension) + ".csv"
            if not os.path.isfile(file_path):
                print(file_path + "not found")
                continue
            raw_data[file] = pd.read_csv(file_path, ";")

        # split by time in seperate lists
        time_filtered_data = dict()
        for name in file_names:
            tmp = []
            for k, end in enumerate(raw_data["points"].time):
                start = int(raw_data["fitts"].time[k])
                # create mask for time interval
                mask = (raw_data[name]["time"] > start) & (raw_data[name]["time"] <= end)
                # only return items matching to mask
                tmp.append(raw_data[name].loc[mask])
            time_filtered_data[name] = tmp
        # list to dataframe
        time_filtered_data["points"] = pd.concat(time_filtered_data["points"])

        # scale screen
        time_filtered_data["points"]["x-press"] = time_filtered_data["points"]["x-press"].div(pixels["width"])
        time_filtered_data["points"]["x-circle"] = time_filtered_data["points"]["x-circle"].div(pixels["width"])
        time_filtered_data["points"]["y-press"] = time_filtered_data["points"]["y-press"].div(pixels["height"])
        time_filtered_data["points"]["y-circle"] = time_filtered_data["points"]["y-circle"].div(pixels["height"])

        # filter unique timestamps
        for k, item in enumerate(time_filtered_data["points"]):
            for sensor in sensors:
                time_filtered_data[sensor][k] = time_filtered_data[sensor][k].drop_duplicates(subset="time", keep="last")
        
        # Create Array
        result_interval = []
        print("create-interval", end=' ')
        for k, point in time_filtered_data["points"].iterrows():
            print(k, end=' ', flush=True)
            one_interval = []
            interval = dict()
            for sensor in sensors:
                interval[sensor] = time_filtered_data[sensor][k]

            # keeps index of each sensor
            position = dict()
            # keeps value of last sensor event
            last_values = dict()
            for sensor in sensors:
                position[sensor] = 0
                last_values[sensor] = 0

            # find maximum first timestamp in all sensors
            current_time = -1
            for sensor in sensors:
                if interval[sensor]["time"].iloc[position[sensor]] > current_time:
                    current_time = interval[sensor]["time"].iloc[position[sensor]]
                last_values[sensor] = interval[sensor][:].iloc[position[sensor]]
            
            # print("start", raw_data["fitts"].time.iloc[k])
            # print("first_data", current_time)
            # print("end", time_filtered_data["points"].iloc[k]["time"])

            # iterate as long as time has not reached max
            while current_time <= time_filtered_data["points"].iloc[k]["time"]:
                # iterate over alle sensors to find values before current_time (maybe one sensor is having multiple updates)
                while True:
                    all_valid = True
                    for sensor in sensors:
                        if position[sensor] + 1 < len(interval[sensor]["time"]):
                            if interval[sensor]["time"].iloc[position[sensor] + 1] <= current_time:
                                position[sensor] += 1
                                last_values[sensor] = interval[sensor][:].iloc[position[sensor]]
                                all_valid = False
                    if all_valid:
                        break
                one_interval.append(last_values)
                print("###", k, one_interval[-1]["acc"].x)
                current_time = current_time + event_time_diff
            result_interval.append(one_interval)
        # kepp for better output
        print()
        
        

processing - participant: 1 smartphone: S3Mini
create-interval 0 ### 0 -5.529676
### 0 -5.529676
### 0 -4.094289
### 0 -4.094289
### 0 -4.094289
### 0 -4.164048
### 0 -4.164048
### 0 -4.164048
### 0 -4.164048
### 0 -4.164048
### 0 -4.164048
### 0 -4.164048
### 0 -4.969898000000001
### 0 -4.969898000000001
### 0 -4.8564453
### 0 -4.8564453
### 0 -4.8564453
### 0 -4.8564453
### 0 -4.921363400000001
### 0 -4.921363400000001
### 0 -4.921363400000001
### 0 -4.921363400000001
### 0 -4.3344407
### 0 -4.3344407
### 0 -2.8829231
### 0 -2.8829231
### 0 -2.8829231
### 0 -2.4495704
### 0 -2.4495704
### 0 -2.4495704
### 0 -2.4621644
### 0 -2.4621644
### 0 -2.4621644
### 0 -2.4621644
### 0 -2.7394580000000004
### 0 -2.7394580000000004
### 0 -2.7394580000000004
### 0 -2.4007432
### 0 -2.4007432
### 0 -2.4007432
### 0 -2.466095
### 0 -2.466095
### 0 -2.466095
### 0 -2.466095
### 0 -3.5209156999999998
### 0 -3.5209156999999998
### 0 -3.5209156999999998
### 0 -3.4871328
### 0 -3.4871328
### 0 -3.4871328

### 0 -2.4677777
### 0 -2.4677777
### 0 -2.3089728
### 0 -2.3089728
### 0 -2.3089728
### 0 -2.043877
### 0 -2.043877
### 0 -2.043877
### 0 -1.7034585
### 0 -1.7034585
### 0 -1.7034585
### 0 -1.7034585
### 0 -1.410153
### 0 -1.410153
### 0 -1.410153
### 0 -1.0748262
### 0 -1.0748262
### 0 -1.0748262
### 0 -0.9835461
### 0 -0.9835461
### 0 -0.9835461
### 0 -0.9835461
### 0 -0.28685427
### 0 -0.28685427
### 0 -0.28685427
### 0 0.5582659000000001
### 0 0.5582659000000001
### 0 0.5582659000000001
### 0 1.0576625
### 0 1.0576625
### 0 1.0576625
### 0 1.0576625
### 0 4.0789175
### 0 4.0789175
### 0 4.0789175
### 0 5.0276165
### 0 5.0276165
### 0 5.0276165
### 0 1.9143039000000002
### 0 1.9143039000000002
### 0 1.9143039000000002
### 0 1.9143039000000002
### 0 2.3736794
### 0 2.3736794
### 0 2.3736794
### 0 2.6843626
### 0 2.6843626
### 0 2.6843626
### 0 2.6799047000000003
### 0 2.6799047000000003
### 0 2.6799047000000003
### 0 2.6799047000000003
### 0 2.881338
### 0 2.881338
### 0 2.881338
##

### 0 2.066081
### 0 2.066081
### 0 2.066081
### 0 2.473119
### 0 2.473119
### 0 2.473119
### 0 2.644943
### 0 2.644943
### 0 2.644943
### 0 2.644943
### 0 2.658697
### 0 2.658697
### 0 2.658697
### 0 2.8139706
### 0 2.8139706
### 0 2.8139706
### 0 2.7441777999999997
### 0 2.7441777999999997
### 0 2.7441777999999997
### 0 2.7441777999999997
### 0 2.8297404999999998
### 0 2.8297404999999998
### 0 2.8297404999999998
### 0 2.7176027
### 0 2.7176027
### 0 2.7176027
### 0 2.4848158
### 0 2.4848158
### 0 2.4848158
### 0 2.4848158
### 0 2.2197144
### 0 2.2197144
### 0 2.2197144
### 0 2.113171
### 0 2.113171
### 0 2.113171
### 0 2.0222015
### 0 2.0222015
### 0 2.0222015
### 0 2.0222015
### 0 1.8871793000000001
### 0 1.8871793000000001
### 0 1.8871793000000001
### 0 1.9358159
### 0 1.9358159
### 0 1.9358159
### 0 1.7546164
### 0 1.7546164
### 0 1.7546164
### 0 1.7546164
### 0 1.5856226999999998
### 0 1.5856226999999998
### 0 1.5856226999999998
### 0 1.5266631000000002
### 0 1.5266631000000002
#

### 0 0.37080562
### 0 0.37080562
### 0 0.37080562
### 0 0.3283242
### 0 0.3283242
### 0 0.3283242
### 0 -0.19240856
### 0 -0.19240856
### 0 -0.19240856
### 0 -0.19240856
### 0 -1.01302
### 0 -1.01302
### 0 -1.01302
### 0 -1.7813305
### 0 -1.7813305
### 0 -1.7813305
### 0 -2.2668846
### 0 -2.2668846
### 0 -2.2668846
### 0 -2.2668846
### 0 -2.4867122000000004
### 0 -2.4867122000000004
### 0 -2.4867122000000004
### 0 -2.4885688
### 0 -2.4885688
### 0 -2.4885688
### 0 -2.4261044999999997
### 0 -2.4261044999999997
### 0 -2.4261044999999997
### 0 -2.4261044999999997
### 0 -2.4261044999999997
### 0 -2.1550462
### 0 -2.1550462
### 0 -2.1550462
### 0 -1.8447386000000001
### 0 -1.8447386000000001
### 0 -1.8447386000000001
### 0 -1.8447386000000001
### 0 -1.5708334
### 0 -1.5708334
### 0 -1.5708334
### 0 -1.4710581
### 0 -1.4710581
### 0 -1.4710581
### 0 -1.3615429
### 0 -1.3615429
### 0 -1.3509521
### 0 -1.3509521
### 0 -1.3509521
### 0 -1.3509521
### 0 -1.0158392
### 0 -1.0158392
### 0 -1.0158

1 ### 1 -0.85462546
### 1 -0.85462546
### 1 -0.85462546
### 1 -0.51326084
### 1 -0.51326084
### 1 -0.51326084
### 1 -0.55709505
### 1 -0.55709505
### 1 -0.55709505
### 1 -0.55709505
### 1 -0.19765615
### 1 -0.19765615
### 1 -0.19765615
### 1 0.20948863
### 1 0.20948863
### 1 0.20948863
### 1 0.5044242
### 1 0.5044242
### 1 0.5044242
### 1 0.42803836
### 1 0.42803836
### 1 0.42803836
### 1 0.23524344
### 1 0.23524344
### 1 0.23524344
### 1 0.23524344
### 1 0.11189628
### 1 0.11189628
### 1 0.11189628
### 1 0.06452274
### 1 0.06452274
### 1 0.06452274
### 1 0.06452274
### 1 0.09318185
### 1 0.09318185
### 1 0.09318185
### 1 0.1510216
### 1 0.1510216
### 1 0.1510216
### 1 0.10225785
### 1 0.10225785
### 1 0.10225785
### 1 0.10225785
### 1 0.14225447
### 1 0.14225447
### 1 0.14225447
### 1 0.049388765999999994
### 1 0.049388765999999994
### 1 0.049388765999999994
### 1 0.049388765999999994
### 1 -0.028312682999999998
### 1 -0.028312682999999998
### 1 -0.028312682999999998
### 1 -0.07762205

...

420 ### 420 0.15604055
### 420 0.15604055
### 420 0.15604055
### 420 0.10601258
### 420 0.10601258
### 420 0.10601258
### 420 0.14795029
### 420 0.14795029
### 420 0.12170124
### 420 0.12170124
### 420 0.12170124
### 420 0.12170124
### 420 0.3563403
### 420 0.3563403
### 420 0.3563403
### 420 0.5783669
### 420 0.5783669
### 420 0.5783669
### 420 0.6735312
### 420 0.6735312
### 420 0.6735312
### 420 0.6735312
### 420 0.5133616
### 420 0.5133616
### 420 0.5133616
### 420 0.5133616
### 420 0.34649146
### 420 0.34649146
### 420 0.34649146
### 420 0.22814274
### 420 0.22814274
### 420 0.22814274
### 420 0.09527469
### 420 0.09527469
### 420 0.09527469
### 420 -0.072377086
### 420 -0.072377086
### 420 -0.072377086
### 420 -0.072377086
### 420 -0.3125887
### 420 -0.3125887
### 420 -0.3125887
### 420 -0.3301524
### 420 -0.3301524
### 420 -0.3301524
### 420 -0.3301524
### 420 -0.005194187
### 420 -0.005194187
### 420 -0.005194187
### 420 -0.00021111965000000001
### 420 -0.00021111965000000001
#

### 422 0.2811364
### 422 0.2811364
### 422 0.2811364
### 422 0.5778793000000001
### 422 0.5778793000000001
### 422 0.5778793000000001
### 422 0.5778793000000001
### 422 0.37726940000000003
### 422 0.37726940000000003
### 422 0.37726940000000003
### 422 -0.27058887
### 422 -0.27058887
### 422 -0.27058887
### 422 -0.27058887
### 422 -0.3954792
### 422 -0.3954792
### 422 -0.3954792
### 422 -0.33762145
### 422 -0.33762145
### 422 -0.33762145
### 422 -0.33762145
### 422 -0.22980976
### 422 -0.22980976
### 422 -0.22980976
### 422 -0.3421116
### 422 -0.3421116
### 422 -0.3421116
### 422 -0.24243712
### 422 -0.24243712
### 422 -0.24243712
### 422 -0.3239479
### 422 -0.3239479
### 422 -0.3239479
### 422 -0.3239479
### 422 -0.5186824999999999
### 422 -0.5186824999999999
### 422 -0.5186824999999999
### 422 -1.149987
### 422 -1.149987
### 422 -1.149987
### 422 -1.2396133
### 422 -1.2396133
### 422 -1.2396133
### 422 -1.2396133
### 422 -0.53742766
### 422 -0.53742766
### 422 -0.53742766
### 422 -0

### 425 -0.011680365
### 425 -0.011680365
### 425 -0.011680365
### 425 -0.32532716
### 425 -0.32532716
### 425 -0.32532716
### 425 -0.32532716
### 425 -0.91495776
### 425 -0.91495776
### 425 -0.91495776
### 425 -1.1792862
### 425 -1.1792862
### 425 -1.1792862
### 425 -0.49213909999999994
### 425 -0.49213909999999994
### 425 -0.49213909999999994
### 425 -0.49213909999999994
### 425 -0.101322174
### 425 -0.101322174
### 425 -0.101322174
### 425 -0.12700629
### 425 -0.12700629
### 425 -0.12700629
### 425 -0.072357416
### 425 -0.072357416
### 425 -0.072357416
### 425 0.12254834
### 425 0.12254834
### 425 0.12254834
### 425 0.12254834
### 425 0.0021061897
### 425 0.0021061897
### 425 0.0021061897
### 425 -0.067489624
### 425 -0.067489624
### 425 -0.067489624
### 425 0.001155138
### 425 0.001155138
### 425 0.001155138
### 425 0.001155138
### 425 -0.19324851
### 425 -0.19324851
### 425 -0.19324851
### 425 -0.8421091999999999
### 425 -0.8421091999999999
### 425 -0.8421091999999999
### 425 -0.9

### 427 -0.18688846
### 427 -0.14264393
### 427 -0.14264393
### 427 -0.14264393
### 427 -0.14264393
### 427 0.07362628
### 427 0.07362628
### 427 0.07362628
### 427 0.38842009999999993
### 427 0.38842009999999993
### 427 0.38842009999999993
### 427 0.5685071999999999
### 427 0.5685071999999999
### 427 0.5685071999999999
### 427 0.69185233
### 427 0.69185233
### 427 0.69185233
### 427 0.69185233
### 427 0.5948458000000001
### 427 0.5948458000000001
### 427 0.5948458000000001
### 427 0.3366897
### 427 0.3366897
### 427 0.3366897
### 427 -0.40900612
### 427 -0.40900612
### 427 -0.40900612
### 427 -0.40900612
### 427 -0.40900612
428 ### 428 0.20956922
### 428 0.20956922
### 428 0.20956922
### 428 0.20956922
### 428 0.20956922
### 428 0.20956922
### 428 0.09156871
### 428 0.09156871
### 428 0.20264626
### 428 0.20264626
### 428 0.20264626
### 428 0.20264626
### 428 0.40793467
### 428 0.40793467
### 428 0.40793467
### 428 0.56122553
### 428 0.56122553
### 428 0.56122553
### 428 0.70751643
##

430 ### 430 -0.6399467
### 430 -0.6399467
### 430 -0.6399467
### 430 -0.22842026
### 430 -0.22842026
### 430 -0.22842026
### 430 -0.23970318
### 430 -0.23970318
### 430 -0.23970318
### 430 0.16114116
### 430 0.16114116
### 430 0.16114116
### 430 0.16114116
### 430 0.5563688
### 430 0.5563688
### 430 0.5563688
### 430 0.8816016
### 430 0.8816016
### 430 0.8816016
### 430 0.9876448
### 430 0.9876448
### 430 0.9876448
### 430 0.8324468
### 430 0.8324468
### 430 0.8324468
### 430 0.8324468
### 430 0.5630748000000001
### 430 0.5630748000000001
### 430 0.5630748000000001
### 430 0.32424855
### 430 0.32424855
### 430 0.32424855
### 430 0.16721129999999998
### 430 0.16721129999999998
### 430 0.16721129999999998
### 430 0.16721129999999998
### 430 0.11469865
### 430 0.11469865
### 430 0.11469865
### 430 0.11469865
### 430 0.024850845
### 430 0.024850845
### 430 -0.2767427
### 430 -0.2767427
### 430 -0.2767427
### 430 -0.2767427
### 430 -0.26330018
### 430 -0.26330018
### 430 -0.26330018
### 430

In [43]:
        
        for i in range(100):
            print(i, result_interval[0][i]["acc"].x)
        
        

0 -0.5508187
1 -0.5508187
2 -0.5508187
3 -0.5508187
4 -0.5508187
5 -0.5508187
6 -0.5508187
7 -0.5508187
8 -0.5508187
9 -0.5508187
10 -0.5508187
11 -0.5508187
12 -0.5508187
13 -0.5508187
14 -0.5508187
15 -0.5508187
16 -0.5508187
17 -0.5508187
18 -0.5508187
19 -0.5508187
20 -0.5508187
21 -0.5508187
22 -0.5508187
23 -0.5508187
24 -0.5508187
25 -0.5508187
26 -0.5508187
27 -0.5508187
28 -0.5508187
29 -0.5508187
30 -0.5508187
31 -0.5508187
32 -0.5508187
33 -0.5508187
34 -0.5508187
35 -0.5508187
36 -0.5508187
37 -0.5508187
38 -0.5508187
39 -0.5508187
40 -0.5508187
41 -0.5508187
42 -0.5508187
43 -0.5508187
44 -0.5508187
45 -0.5508187
46 -0.5508187
47 -0.5508187
48 -0.5508187
49 -0.5508187
50 -0.5508187
51 -0.5508187
52 -0.5508187
53 -0.5508187
54 -0.5508187
55 -0.5508187
56 -0.5508187
57 -0.5508187
58 -0.5508187
59 -0.5508187
60 -0.5508187
61 -0.5508187
62 -0.5508187
63 -0.5508187
64 -0.5508187
65 -0.5508187
66 -0.5508187
67 -0.5508187
68 -0.5508187
69 -0.5508187
70 -0.5508187
71 -0.5508187
72

In [3]:
        # make numpy arrays (without dicts)
        print("make numpy-array interval")
        final_intervals = []
        print("transpose-interval", end=' ')
        for k, interval in enumerate(result_interval):
            print(k, end=' ', flush=True)
            current_interval = []
            for i in interval:
                event = []
                for sensor in i.values():
                    # except timestamp
                    for value in sensor[1:]:
                        event.append(value)
                current_interval.append(np.asarray(event))
            # make list to array, transpose and make 2d matrix
            final_intervals.append(np.array(np.asarray(current_interval).transpose()))
            
        print(len(final_intervals))

        print("make numpy-array points")
        final_points = time_filtered_data["points"][['x-press','x-circle','y-press','y-circle']].values
        final_result = [final_points, final_intervals]

        # save dump pickles
        data_path = str(Path.home()) + "/data/pickles/fapra_imu-processed-" +  str(PID) + "-" + smartphone + ".pkl"
        pkl.dump(final_result, open( data_path, "wb" ))

processing - participant: 1 smartphone: S3Mini
create-interval 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261